In [23]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import dash

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import CRUD module for database operations
from animal_shelter import AnimalShelter

# Data Manipulation / Model
username = "aacuser"
password = "Girginzoom1"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Retrieve all data from the database and load it into a DataFrame
df = pd.DataFrame.from_records(db.read({}))

# Check if '_id' column exists before trying to drop it
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Print column names for debugging
print("DataFrame columns:", df.columns.tolist())
print("DataFrame shape:", df.shape)

# Dashboard Layout / View
app = JupyterDash(__name__)

# Load an image for the dashboard
try:
    image_filename = 'Grazioso Salvare Logo.png' 
    encoded_image = base64.b64encode(open(image_filename, 'rb').read())
    image_element = html.Div([
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                'width': '300px',
                'height': 'auto',
                'display': 'block',
                'margin': '0 auto',
                'padding': '20px',
                'border': '1px solid #ccc',  # Add a light gray border
                'borderRadius': '5px',  # Round the corners
                'boxShadow': '0 4px 8px rgba(0,0,0,0.1)'  # Add a subtle shadow
            }
        )
    ], style={
        'textAlign': 'center',
        'width': '100%',
        'margin': '20px 0'
    })
    
except FileNotFoundError:
    print("Logo image not found. Continuing without it.")
    image_element = html.Div()

# Callback for filter dropdown and buttons
@app.callback(
    [Output('datatable-id', 'data'),
     Output('graph-id', 'children'),
     Output('map-id', 'children'),
     Output('filter-details', 'children')],
    [Input('filter-type', 'value'),
     Input('button-one', 'n_clicks'),
     Input('button-two', 'n_clicks'),
     Input('button-three', 'n_clicks'),
     Input('button-four', 'n_clicks')],
    prevent_initial_call=True
)
def update_filters(rescue_type, button1, button2, button3, button4):
    # Get the triggered input
    ctx = dash.callback_context
    if not ctx.triggered:
        trigger_id = 'filter-type'
    else:
        trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # Handle button clicks first
    if trigger_id == 'button-one':
        filtered_df = pd.DataFrame.from_records(db.read({
            '$and': [
                {'$or': [
                    {'breed': {'$regex': 'Labrador Retriever', '$options': 'i'}},
                    {'breed': {'$regex': 'Chesapeake Bay Retriever', '$options': 'i'}},
                    {'breed': {'$regex': 'Newfoundland', '$options': 'i'}}
                ]},
                {'sex_upon_outcome': 'Intact Female'},
                {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
            ]
        }))
        filter_details = html.Div([
            html.P("Water Rescue Criteria:"),
            html.P("Breeds: Labrador Retriever, Chesapeake Bay Retriever, Newfoundland"),
            html.P("Gender: Intact Female"),
            html.P("Age: 26-156 weeks")
        ])
    elif trigger_id == 'button-two':
        filtered_df = pd.DataFrame.from_records(db.read({
            '$and': [
                {'$or': [
                    {'breed': {'$regex': 'German Shepherd', '$options': 'i'}},
                    {'breed': {'$regex': 'Alaskan Malamute', '$options': 'i'}},
                    {'breed': {'$regex': 'Old English Sheepdog', '$options': 'i'}},
                    {'breed': {'$regex': 'Siberian Husky', '$options': 'i'}},
                    {'breed': {'$regex': 'Rottweiler', '$options': 'i'}}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
            ]
        }))
        filter_details = html.Div([
            html.P("Mountain Rescue Criteria:"),
            html.P("Breeds: German Shepherd, Alaskan Malamute, Old English Sheepdog, Siberian Husky, Rottweiler"),
            html.P("Gender: Intact Male"),
            html.P("Age: 26-156 weeks")
        ])
    elif trigger_id == 'button-three':
        filtered_df = pd.DataFrame.from_records(db.read({
            '$and': [
                {'$or': [
                    {'breed': {'$regex': 'Doberman Pinscher', '$options': 'i'}},
                    {'breed': {'$regex': 'German Shepherd', '$options': 'i'}},
                    {'breed': {'$regex': 'Golden Retriever', '$options': 'i'}},
                    {'breed': {'$regex': 'Bloodhound', '$options': 'i'}},
                    {'breed': {'$regex': 'Rottweiler', '$options': 'i'}}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}}
            ]
        }))
        filter_details = html.Div([
            html.P("Disaster Rescue Criteria:"),
            html.P("Breeds: Doberman Pinscher, German Shepherd, Golden Retriever, Bloodhound, Rottweiler"),
            html.P("Gender: Intact Male"),
            html.P("Age: 20-300 weeks")
        ])
    elif trigger_id == 'button-four':
        filtered_df = pd.DataFrame.from_records(db.read({}))
        filter_details = html.P("Showing all animals")
    # If no button was clicked, use the dropdown value
    elif rescue_type == 'water':
        filtered_df = df[
            (df['breed'].str.contains('Labrador Retriever|Golden Retriever|Newfoundland|Portuguese Water Dog|English Setter|Irish Setter|Nova Scotia Duck Tolling Retriever|Curly-Coated Retriever|American Water Spaniel|Chesapeake Bay Retriever|Spanish Water Dog|Barbet|Lagotto Romagnolo|Xoloitzcuintli', case=False, na=False)) &
            (df['sex_upon_outcome'] == 'Intact Female') &
            (df['age_upon_outcome_in_weeks'] >= 26) &
            (df['age_upon_outcome_in_weeks'] <= 156)
        ]
        filter_details = html.Div([
            html.P("Breeds: Labrador Retriever, Golden Retriever, Newfoundland, Portuguese Water Dog, English Setter, Irish Setter, Nova Scotia Duck Tolling Retriever, Curly-Coated Retriever, American Water Spaniel, Chesapeake Bay Retriever, Spanish Water Dog, Barbet, Lagotto Romagnolo, Xoloitzcuintli"),
            html.P("Gender: Intact Female"),
            html.P("Age: 26-156 weeks")
        ])
    elif rescue_type == 'mountain':
        filtered_df = df[
            (df['breed'].str.contains('Bernese Mountain Dog|Greater Swiss Mountain Dog|Cane Corso|Doberman Pinscher|German Shorthaired Pointer|Rhodesian Ridgeback|Bloodhound|Border Collie|Belgian Malinois|Australian Cattle Dog|Australian Shepherd|Australian Kelpie|Alaskan Malamute|Siberian Husky|Samoyed|Akita|Alaskan Husky|American Eskimo Dog|Chinook|Keeshond|Norwegian Elkhound|Xoloitzcuintli', case=False, na=False)) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'] >= 26) &
            (df['age_upon_outcome_in_weeks'] <= 156)
        ]
        filter_details = html.Div([
            html.P("Breeds: Bernese Mountain Dog, Greater Swiss Mountain Dog, Cane Corso, Doberman Pinscher, German Shorthaired Pointer, Rhodesian Ridgeback, Bloodhound, Border Collie, Belgian Malinois, Australian Cattle Dog, Australian Shepherd, Australian Kelpie, Alaskan Malamute, Siberian Husky, Samoyed, Akita, Alaskan Husky, American Eskimo Dog, Chinook, Keeshond, Norwegian Elkhound, Xoloitzcuintli"),
            html.P("Gender: Intact Male"),
            html.P("Age: 26-156 weeks")
        ])
    elif rescue_type == 'disaster':
        filtered_df = df[
            (df['breed'].str.contains('Doberman Pinscher|German Shorthaired Pointer|Golden Retriever|Bloodhound|Border Collie|Belgian Malinois|Labrador Retriever|Australian Cattle Dog|Australian Shepherd|Australian Kelpie|Alaskan Malamute|Siberian Husky|Samoyed|Akita|Alaskan Husky|American Eskimo Dog|Chinook|Keeshond|Norwegian Elkhound|Xoloitzcuintli', case=False, na=False)) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'] >= 20) &
            (df['age_upon_outcome_in_weeks'] <= 300)
        ]
        filter_details = html.Div([
            html.P("Breeds: Doberman Pinscher, German Shorthaired Pointer, Golden Retriever, Bloodhound, Border Collie, Belgian Malinois, Labrador Retriever, Australian Cattle Dog, Australian Shepherd, Australian Kelpie, Alaskan Malamute, Siberian Husky, Samoyed, Akita, Alaskan Husky, American Eskimo Dog, Chinook, Keeshond, Norwegian Elkhound, Xoloitzcuintli"),
            html.P("Gender: Intact Male"),
            html.P("Age: 20-300 weeks")
        ])
    else:  # ALL or default case
        filtered_df = df
        filter_details = html.P("Showing all animals")

    # Create the graph
    if len(filtered_df) == 0:
        graph = html.Div("No animals match the selected criteria.")
    else:
        breed_counts = filtered_df['breed'].value_counts().head(10)
        fig = px.bar(
            x=breed_counts.index,
            y=breed_counts.values,
            title='Animals by Breed',
            labels={'x': 'Breed', 'y': 'Count'}
        )
        fig.update_layout(
            xaxis_tickangle=-45,
            height=500
        )
        graph = dcc.Graph(figure=fig)
    
    # Create the map
    if len(filtered_df) == 0:
        map_view = html.Div("No animals match the selected criteria.")
    else:
        try:
            # Try to find latitude and longitude columns
            lat_col = None
            lon_col = None
            
            # Look for common latitude/longitude column names
            for col in filtered_df.columns:
                if 'lat' in col.lower():
                    lat_col = col
                elif 'lon' in col.lower() or 'long' in col.lower():
                    lon_col = col
            
            # If we found the columns, create the map
            if lat_col and lon_col:
                map_view = px.scatter_mapbox(
                    filtered_df,
                    lat=lat_col,
                    lon=lon_col,
                    hover_name='breed',
                    hover_data=['age_upon_outcome', 'sex_upon_outcome'],
                    zoom=10,
                    title='Animal Locations'
                )
                map_view.update_layout(
                    mapbox_style='open-street-map',
                    height=500
                )
                map_view = dcc.Graph(figure=map_view)
            else:
                # If we couldn't find the columns, show a message
                map_view = html.Div("Location data not available in the dataset.")
        except Exception as e:
            print(f"Error creating map: {str(e)}")
            map_view = html.Div("Error creating map visualization.")
    
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)
    
    return filtered_df.to_dict('records'), graph, map_view, filter_details

# Define the layout
app.layout = html.Div([
    # Centered title
    html.Center(html.B(html.H1('Kevin Caverly CS-340 Dashboard'))),
    html.Hr(),

    image_element,

    # Filter dropdown
    html.Div([
        html.H3("Filter Animals by Rescue Type"),
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'ALL'}
            ],
            value='ALL',
            clearable=False,
            style={'width': '50%'}
        ),
        html.Div([
            html.Div(id='filter-details', className="mt-2")
        ])
    ], className="p-3 bg-light rounded mb-4"),
    
    # Filter buttons
    html.Div([
        html.H3("Quick Filters"),
        html.Div([
            html.Button('Water Rescue', id='button-one', n_clicks=0, className='btn btn-primary m-2'),
            html.Button('Mountain Rescue', id='button-two', n_clicks=0, className='btn btn-primary m-2'),
            html.Button('Disaster Rescue', id='button-three', n_clicks=0, className='btn btn-primary m-2'),
            html.Button('Reset', id='button-four', n_clicks=0, className='btn btn-secondary m-2')
        ], className='d-flex justify-content-center')
    ], className="p-3 bg-light rounded mb-4"),
    
    # Data table
    html.Div([
        html.H3("Animal Data"),
        dash_table.DataTable(
            id='datatable-id',
            columns=[
                {"name": i, "id": i} for i in df.columns
            ],
            data=df.to_dict('records'),
            page_size=10,
            style_table={'overflowX': 'auto'},
            style_cell={
                'height': 'auto',
                'minWidth': '100px', 'width': 'auto', 'maxWidth': '180px',
                'whiteSpace': 'normal',
                'textAlign': 'left'
            }
        )
    ], className="mb-4"),
    
    # Graph
    html.Div([
        html.H3("Animal Distribution by Breed"),
        html.Div(id='graph-id')
    ], className="mb-4"),
    
    # Map
    html.Div([
        html.H3("Animal Locations"),
        html.Div(id='map-id')
    ], className="mb-4")
])

# Run the app server
app.run_server(debug=True) 

DataFrame columns: ['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color', 'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long', 'age_upon_outcome_in_weeks', 'species', 'age']
DataFrame shape: (10055, 18)
Dash app running on http://127.0.0.1:15289/
